# Evaluating Hyperparameters- Deep Learning

### 1) Importing Libraries and Loading Data

In [1]:
import os
import time
import operator
import warnings
from math import floor

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import bayes_opt 

from numpy import argmax, unique, reshape

from scipy.stats import randint

from sklearn import datasets, metrics, tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    StratifiedKFold,
    cross_val_score
)
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.tree import plot_tree

from bayes_opt import BayesianOptimization

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Dropout,
    Conv1D,
    Conv2D,
    BatchNormalization,
    Flatten,
    MaxPooling1D,
    LeakyReLU
)
from tensorflow.keras.optimizers import (
    Adam,
    SGD,
    RMSprop,
    Adadelta,
    Adagrad,
    Adamax,
    Nadam,
    Ftrl
)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.utils import to_categorical

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)


In [2]:
#Create a path to where your data is stored.
path = r'C:\Users\wjzak\Dropbox\Career Foundry\Data Specialization\Module 2\01-2026- Analysis\01 Data'

In [3]:
#Read in the Regular weather data.
climate = pd.read_csv(os.path.join(path, 'climate_cleaned.csv'))
climate

,DATE,MONTH,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,SONNBLICK_cloud_cover,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max,KASSEL_cloud_cover,STOCKHOLM_humidity,MUNCHENB_pressure
0,19600101,1,7,0.85,1.0180,0.32,0.09,0.7,6.5,0.8,10.9,1,0.81,1.0195,0.88,0.00,7.0,3.7,-0.9,7.9,4,0.67,1.017,0.44,0.01,2.3,2.4,-0.4,5.1,7,0.85,1.0032,0.07,0.25,0.0,9.3,7.4,11.0,8,0.83,1.0161,0.12,0.08,0.0,10.0,7.0,11.5,7,0.91,1.0010,0.13,0.22,0.0,10.6,9.4,8.3,0.82,1.0094,0.28,0.48,1.6,7.9,3.9,9.4,8,1.00,1.0173,0.20,0.00,0.0,-0.6,-1.9,0.5,7,0.83,1.0063,0.22,0.32,1.0,9.5,8.5,11.1,6,0.92,1.0260,0.53,0.00,1.4,7.6,4.4,10.8,5,0.67,0.20,0.10,0.0,6.9,1.1,10.4,8,0.98,0.9978,0.04,1.14,0.0,4.9,3.8,5.9,4,0.73,1.0304,0.48,0.01,2.3,-5.9,-8.5,-3.2,5,1.0114,0.05,0.32,0.0,4.2,2.2,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9,8,0.98,1.0304
1,19600102,1,6,0.84,1.0180,0.36,1.05,1.1,6.1,3.3,10.1,6,0.84,1.0172,0.25,0.00,0.0,2.9,2.2,4.4,4,0.67,1.017,0.18,0.31,0.0,2.3,1.4,3.1,8,0.90,1.0056,0.14,0.06,0.1,7.7,6.4,8.3,8,0.89,1.0161,0.18,0.66,0.5,8.2,7.4,11.0,7,0.98,1.0051,0.13,0.23,0.0,6.1,3.9,10.6,0.86,1.0086,0.12,0.27,0.0,7.7,6.8,9.1,6,0.94,1.0173,0.56,0.13,3.2,2.1,-1.3,5.5,8,0.92,1.0062,0.17,1.34,0.4,8.6,7.5,9.9,7,0.86,1.0254,0.46,0.00,0.9,9.8,7.4,12.2,6,0.72,0.61,0.30,5.1,6.2,4.2,10.2,8,0.62,1.0139,0.04,0.00,0.0,3.4,2.8,4.9,6,0.97,1.0292,0.21,0.61,0.0,-9.5,-10.5,-8.5,5,1.0114,0.05,0.06,0.0,4.0,3.0,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1,6,0.62,1.0292
2,19600103,1,8,0.90,1.0180,0.18,0.30,0.0,8.5,5.1,9.9,6,0.77,1.0179,0.67,0.00,3.5,3.1,-0.5,6.4,4,0.67,1.017

In [4]:
#Read in the Pleasant weather data.
pleasant = pd.read_csv(os.path.join(path, 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))
pleasant

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,20221027,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22946,20221028,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22947,20221029,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22948,20221030,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 2) Data Cleaning for Exercise

In [5]:
climate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22950 entries, 0 to 22949
Columns: 137 entries, DATE to MUNCHENB_pressure
dtypes: float64(120), int64(17)
memory usage: 24.0 MB


In [6]:
pleasant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22950 entries, 0 to 22949
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   DATE                         22950 non-null  int64
 1   BASEL_pleasant_weather       22950 non-null  int64
 2   BELGRADE_pleasant_weather    22950 non-null  int64
 3   BUDAPEST_pleasant_weather    22950 non-null  int64
 4   DEBILT_pleasant_weather      22950 non-null  int64
 5   DUSSELDORF_pleasant_weather  22950 non-null  int64
 6   HEATHROW_pleasant_weather    22950 non-null  int64
 7   KASSEL_pleasant_weather      22950 non-null  int64
 8   LJUBLJANA_pleasant_weather   22950 non-null  int64
 9   MAASTRICHT_pleasant_weather  22950 non-null  int64
 10  MADRID_pleasant_weather      22950 non-null  int64
 11  MUNCHENB_pleasant_weather    22950 non-null  int64
 12  OSLO_pleasant_weather        22950 non-null  int64
 13  SONNBLICK_pleasant_weather   22950 non-null  i

In [7]:
climate.shape

(22950, 137)

In [8]:
pleasant.shape

(22950, 16)

In [9]:
# Drop columns starting with 'GDANSK', 'ROMA', or 'TOURS' from climate dataset
# These are not available in the pleasant dataset
climate = climate.drop(columns=[col for col in climate.columns if col.startswith(('GDANSK', 'ROMA', 'TOURS'))])

In [10]:
# Extract observation types by city and show missingness
from collections import defaultdict

# Dictionary to store observation types per city
city_observations = defaultdict(set)

# Parse column names to extract city and observation type
for col in climate.columns:
    if col not in ['DATE', 'MONTH']:
        # Split by underscore - everything before the last part is city, last part(s) is observation
        parts = col.rsplit('_', 1)
        if len(parts) == 2:
            city = parts[0]
            obs_type = parts[1]
            city_observations[city].add(obs_type)

# Create summary dataframe showing available observations per city
cities = sorted(city_observations.keys())
obs_types = sorted(set().union(*city_observations.values()))

# Create a matrix showing availability (1) and missingness (0) for each observation type per city
availability_matrix = []
for city in cities:
    row = [city]
    for obs in obs_types:
        if obs in city_observations[city]:
            # Check if this observation has missing values
            col_name = f"{city}_{obs}"
            missing_pct = (climate[col_name].isna().sum() / len(climate)) * 100
            row.append(f"✓ ({missing_pct:.1f}%)" if missing_pct == 0 else f"✗ ({missing_pct:.1f}%)")
        else:
            row.append("N/A")
    availability_matrix.append(row)

# Create dataframe
summary_df = pd.DataFrame(availability_matrix, columns=['City'] + obs_types)

print("OBSERVATION TYPE AVAILABILITY BY CITY (with % missing values)")
print("=" * 150)
print(summary_df.to_string(index=False))

# Summary statistics
print("\n\nSUMMARY BY OBSERVATION TYPE")
print("=" * 60)
for obs in obs_types:
    cities_with_obs = sum(1 for city in cities if obs in city_observations[city])
    print(f"{obs:20} - Available in {cities_with_obs}/{len(cities)} cities")

OBSERVATION TYPE AVAILABILITY BY CITY (with % missing values)
             City    cover humidity      max     mean      min precipitation pressure radiation sunshine
            BASEL      N/A ✓ (0.0%)      N/A      N/A      N/A      ✓ (0.0%) ✓ (0.0%)       N/A ✓ (0.0%)
      BASEL_cloud ✓ (0.0%)      N/A      N/A      N/A      N/A           N/A      N/A       N/A      N/A
     BASEL_global      N/A      N/A      N/A      N/A      N/A           N/A      N/A  ✓ (0.0%)      N/A
       BASEL_temp      N/A      N/A ✓ (0.0%) ✓ (0.0%) ✓ (0.0%)           N/A      N/A       N/A      N/A
         BELGRADE      N/A ✓ (0.0%)      N/A      N/A      N/A      ✓ (0.0%) ✓ (0.0%)       N/A ✓ (0.0%)
   BELGRADE_cloud ✓ (0.0%)      N/A      N/A      N/A      N/A           N/A      N/A       N/A      N/A
  BELGRADE_global      N/A      N/A      N/A      N/A      N/A           N/A      N/A  ✓ (0.0%)      N/A
    BELGRADE_temp      N/A      N/A ✓ (0.0%) ✓ (0.0%) ✓ (0.0%)           N/A      N/A       N/A   

In [11]:
climate.shape

(22950, 137)

In [12]:
# Drop wind_speed and snow_depth columns
climate = climate.drop(columns=[col for col in climate.columns if 'wind_speed' in col.lower() or 'snow_depth' in col.lower()])
print(f"Climate shape after removing wind_speed and snow_depth: {climate.shape}")

Climate shape after removing wind_speed and snow_depth: (22950, 137)


In [13]:
# 1. Define the 15 cities and the specific order of the 9 measurements
expected_cities = [
    'BASEL', 'BELGRADE', 'BUDAPEST', 'DEBILT', 'DUSSELDORF', 
    'HEATHROW', 'KASSEL', 'LJUBLJANA', 'MAASTRICHT', 'MADRID', 
    'MUNCHENB', 'OSLO', 'SONNBLICK', 'STOCKHOLM', 'VALENTIA'
]

measurement_order = [
    'cloud_cover', 'humidity', 'pressure', 'global_radiation', 
    'precipitation', 'sunshine', 'temp_mean', 'temp_min', 'temp_max'
]

# 2. Define relationships for filling
proximity_pairs = {
    'KASSEL': 'LJUBLJANA',
    'STOCKHOLM': 'OSLO',
    'MUNCHENB': 'SONNBLICK'
}

# 3. Enhanced Filling Logic: Create missing columns and fill from neighbors
for city1, city2 in proximity_pairs.items():
    print(f"\nProcessing Pair: {city1} <-> {city2}")
    
    for meas in measurement_order:
        col1 = f"{city1}_{meas}"
        col2 = f"{city2}_{meas}"
        
        # If col1 is missing from the dataframe entirely, create it
        if col1 not in climate.columns:
            print(f"  -> Creating missing column: {col1}")
            # Use neighbor data if available, otherwise NaN
            climate[col1] = climate[col2] if col2 in climate.columns else np.nan
            
        # If col2 is missing from the dataframe entirely, create it
        if col2 not in climate.columns:
            print(f"  -> Creating missing column: {col2}")
            climate[col2] = climate[col1] if col1 in climate.columns else np.nan

        # Now that both columns exist, fill specific NaNs from each other
        climate[col1] = climate[col1].fillna(climate[col2])
        climate[col2] = climate[col2].fillna(climate[col1])

# 4. Global Imputation (Safety Net)
# If both cities in a pair were missing the same measurement, columns will still have NaNs
climate = climate.fillna(climate.mean(numeric_only=True))

# 5. Force the exact 135 columns in the exact order
# This guarantees your X shape is (22950, 135)
expected_columns = [f"{city}_{meas}" for city in expected_cities for meas in measurement_order]

# Ensure every expected column exists (fill with 0 if somehow still missing)
for col in expected_columns:
    if col not in climate.columns:
        climate[col] = 0.0

X = climate[expected_columns]
y = pleasant.drop(columns=['DATE'])

# Reshape for Keras
X_reshaped = X.values.reshape(X.shape[0], X.shape[1], 1)

print(f"\nFinal Validation:")
print(f"X shape: {X.shape}") # (22950, 135)
print(f"X_reshaped shape: {X_reshaped.shape}") # (22950, 135, 1)


Processing Pair: KASSEL <-> LJUBLJANA

Processing Pair: STOCKHOLM <-> OSLO

Processing Pair: MUNCHENB <-> SONNBLICK

Final Validation:
X shape: (22950, 135)
X_reshaped shape: (22950, 135, 1)


In [14]:
# Re-extract features and target after filling
X = climate.drop(columns=['DATE', 'MONTH'])
y = pleasant.drop(columns=['DATE'])

# Reshape for Keras
X_reshaped = X.values.reshape(-1,15,9)
y_reshaped = y.values

print(f"Updated X shape: {X.shape}")
print(f"Updated X_reshaped shape: {X_reshaped.shape}")
print(f"Updated y shape: {y.shape}")

Updated X shape: (22950, 135)
Updated X_reshaped shape: (22950, 15, 9)
Updated y shape: (22950, 15)


In [15]:
# Export cleaned data to CSV files
#climate.to_csv(os.path.join(path, 'climate_cleaned.csv'), index=False)
#y.to_csv(os.path.join(path, 'pleasant_weather_cleaned.csv'), index=False)

# Optionally, save the reshaped arrays as numpy files for later use
#np.save(os.path.join(path, 'X_reshaped.npy'), X_reshaped)
#np.save(os.path.join(path, 'y_reshaped.npy'), y_reshaped)

print("Data export complete!")
print(f"Saved files:")
print(f"  - climate_cleaned.csv ({climate.shape[0]} rows, {climate.shape[1]} columns)")
print(f"  - pleasant_weather_cleaned.csv ({y.shape[0]} rows, {y.shape[1]} columns)")
print(f"  - X_reshaped.npy (shape: {X_reshaped.shape})")
print(f"  - y_reshaped.npy (shape: {y_reshaped.shape})")

Data export complete!
Saved files:
  - climate_cleaned.csv (22950 rows, 137 columns)
  - pleasant_weather_cleaned.csv (22950 rows, 15 columns)
  - X_reshaped.npy (shape: (22950, 15, 9))
  - y_reshaped.npy (shape: (22950, 15))


In [16]:
# Use argmax to transform y

y =  np.argmax(y, axis = 1)
y

array([0, 0, 0, ..., 0, 0, 0], shape=(22950,))

In [17]:
# Check y layout

from sklearn.utils.multiclass import type_of_target
type_of_target(y)

'multiclass'

### 3) Convolutional Neural Networks- Optimization

In [18]:
stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'
}

In [19]:
# Split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 135) (17212,)
(5738, 135) (5738,)


In [20]:
timesteps = X_train.shape[1]
input_dim = 1
n_classes = 15  # Number of weather stations

score_acc = make_scorer(accuracy_score)

In [25]:
# Create function
def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]
    optimizer_name = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)

    nn_opt = BayesianOptimization(bay_area, params, random_state=42)
    nn_opt.maximize(init_points=15, n_iter=4)
    
    # Manual K-Fold cross-validation to avoid sklearn compatibility issues
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    scores = []
    
    for train_idx, val_idx in kfold.split(X_train, y_train):
        X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]
        
        # Create a fresh optimizer for each fold to avoid variable conflicts
        optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
                     'RMSprop':RMSprop(learning_rate=learning_rate), 'Adadelta':Adadelta(learning_rate=learning_rate),
                     'Adagrad':Adagrad(learning_rate=learning_rate), 'Adamax':Adamax(learning_rate=learning_rate),
                     'Nadam':Nadam(learning_rate=learning_rate), 'Ftrl':Ftrl(learning_rate=learning_rate)}
        opt = optimizerD[optimizer_name]
        
        # Build model
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))
        
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax'))
        model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
        
        # Train with early stopping
        es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=0, patience=10, restore_best_weights=True)
        model.fit(X_fold_train.values.reshape(-1, timesteps, input_dim), y_fold_train, 
                  validation_data=(X_fold_val.values.reshape(-1, timesteps, input_dim), y_fold_val),
                  epochs=epochs, batch_size=batch_size, verbose=0, callbacks=[es])
        
        # Evaluate
        _, acc = model.evaluate(X_fold_val.values.reshape(-1, timesteps, input_dim), y_fold_val, verbose=0)
        scores.append(acc)
    
    return np.mean(scores)

In [26]:
start = time.time()
params ={
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation':(0, 9), #9
    'optimizer':(0,7), #7
    'learning_rate':(0.01, 1),
    'batch_size': (200, 1000), #(10, 50), #
    'epochs':(20, 100),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) #25
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   |  neurons  |  kernel   | activa... | optimizer | learni... | batch_... |  epochs   |  layers1  |  layers2  | normal... |  dropout  | dropou... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|   iter    |  target   |  neurons  |  kernel   | activa... | optimizer | learni... | batch_... |  epochs   |  layers1  |  layers2  | normal... |  dropout  | dropou... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|   iter    |  target   |  neurons  |  kernel   | activa... | optimizer | learni... | batch_... |  epochs   |  layers1  |  layers2  | normal... |  dropout  | dropou... |
------------------------------------------------------------------------------------------------------------------------------------------------------


KeyboardInterrupt



In [27]:
optimum = nn_opt.max['params']
learning_rate = optimum['learning_rate']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
optimum['activation'] = activationL[round(optimum['activation'])]

optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'Adam']
optimizerD = {
    'Adam': Adam(learning_rate=learning_rate),
    'SGD': SGD(learning_rate=learning_rate),
    'RMSprop': RMSprop(learning_rate=learning_rate),
    'Adadelta': Adadelta(learning_rate=learning_rate),
    'Adagrad': Adagrad(learning_rate=learning_rate),
    'Adamax': Adamax(learning_rate=learning_rate),
    'Nadam': Nadam(learning_rate=learning_rate),
    'Ftrl': Ftrl(learning_rate=learning_rate)
}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]
optimum

TypeError: 'NoneType' object is not subscriptable

In [24]:
start = time.time()
params ={
    'neurons': (30, 45),  # Centered around 37
    'kernel': (1, 2),  # Kernel was ~1.2, keep lower range
    'activation':(5, 7),  # elu is at index 6, explore nearby activations
    'optimizer':(2, 4),  # Adadelta was selected, explore nearby
    'learning_rate':(0.08, 0.18),  # Centered around 0.13
    'batch_size': (500, 700),  # Centered around 596
    'epochs':(20, 30),  # Centered around 23
    'layers1':(2, 3),  # Keep at 2-3
    'layers2':(1, 3),  # Keep at 1-3
    'normalization':(0.5, 1),  # Was 0.66, keep normalization on
    'dropout':(0, 0.5),  # Was 0.31, explore lower dropout
    'dropout_rate':(0.1, 0.2)  # Centered around 0.156
}
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) #25
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   |  neurons  |  kernel   | activa... | optimizer | learni... | batch_... |  epochs   |  layers1  |  layers2  | normal... |  dropout  | dropou... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------


KeyboardInterrupt: 

Took about 45 minutes to run- had to restart kernel so data will not be saved for this section.

In [46]:
optimum = nn_opt.max['params']
learning_rate = optimum['learning_rate']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
optimum['activation'] = activationL[round(optimum['activation'])]

optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'Adam']
optimizerD = {
    'Adam': Adam(learning_rate=learning_rate),
    'SGD': SGD(learning_rate=learning_rate),
    'RMSprop': RMSprop(learning_rate=learning_rate),
    'Adadelta': Adadelta(learning_rate=learning_rate),
    'Adagrad': Adagrad(learning_rate=learning_rate),
    'Adamax': Adamax(learning_rate=learning_rate),
    'Nadam': Nadam(learning_rate=learning_rate),
    'Ftrl': Ftrl(learning_rate=learning_rate)
}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]
optimum

{'neurons': 40,
 'kernel': np.float64(1.4947943549418734),
 'activation': 'selu',
 'optimizer': <keras.src.optimizers.adadelta.Adadelta at 0x1e232b4a5d0>,
 'learning_rate': np.float64(0.17671185085126348),
 'batch_size': 640,
 'epochs': 28,
 'layers1': 3,
 'layers2': 1,
 'normalization': np.float64(0.9161456638353741),
 'dropout': np.float64(0.35970803504110366),
 'dropout_rate': np.float64(0.16478878438733888)}

In [73]:
# Set the model with optimized hyperparameters
from tensorflow.keras import backend as K

# Clear any previous model/session
K.clear_session()

epochs = 28
batch_size = 640

# Convert to reshaped array to get dimensions
X_train_temp = X_train.values.reshape(-1, 15, 9)
timesteps = len(X_train_temp[0])
input_dim = len(X_train_temp[0][0])
n_classes = 15  # match weather station count

layers1 = 3
layers2 = 1
activation = 'selu'
kernel = int(round(1.4947943549418734))  # Rounded kernel size for Conv1D
neurons = 40
normalization = 0.9161456638353741
dropout = 0.35970803504110366
dropout_rate = 0.16478878438733888
optimizer_lr = 0.17671185085126348

model = Sequential()
model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))

if normalization > 0.5:
    model.add(BatchNormalization())

for i in range(layers1):
    model.add(Dense(neurons, activation=activation))

if dropout > 0.5:
    model.add(Dropout(dropout_rate))

for i in range(layers2):
    model.add(Dense(neurons, activation=activation))

model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) 

# Use categorical_crossentropy since we'll use one-hot encoded labels
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=optimizer_lr), metrics=['accuracy'])

In [57]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 15, 40)         │           400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 15, 40)         │           160 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15, 40)         │         1,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15, 40)         │         1,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 15, 40)         │         1,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15, 40)         │         1,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 7, 40)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 280)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 15)             │         4,215 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,335 (44.28 KB)

 Trainable params: 11,255 (43.96 KB)

 Non-trainable params: 80 (320.00 B)

In [82]:
# Put the y_test set back into a one-hot configuration

y_train_one_hot = to_categorical(y_train)
print(y_train_one_hot)

[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [83]:
# Check shapes

print(f'X_train shape: {X_train.shape}')
print(f'y_train_one_hot shape: {y_train_one_hot.shape}')

X_train shape: (17212, 135)
y_train_one_hot shape: (17212, 15)


In [84]:
# Fit the model to the data
# Reshape X_train from (samples, 135) to (samples, 15, 9)
X_train_reshaped = X_train.values.reshape(-1, 15, 9)

model.fit(X_train_reshaped, y_train_one_hot, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/28
27/27 - 0s - 9ms/step - accuracy: 0.4902 - loss: 29.9892
Epoch 2/28
27/27 - 0s - 9ms/step - accuracy: 0.4389 - loss: 26.1835
Epoch 3/28
27/27 - 0s - 10ms/step - accuracy: 0.4602 - loss: 36.3128
Epoch 4/28
27/27 - 0s - 9ms/step - accuracy: 0.4446 - loss: 35.6664
Epoch 5/28
27/27 - 0s - 9ms/step - accuracy: 0.4710 - loss: 35.2437
Epoch 6/28
27/27 - 0s - 9ms/step - accuracy: 0.4503 - loss: 30.7016
Epoch 7/28
27/27 - 0s - 9ms/step - accuracy: 0.4780 - loss: 27.2136
Epoch 8/28
27/27 - 0s - 9ms/step - accuracy: 0.4529 - loss: 29.8785
Epoch 9/28
27/27 - 0s - 9ms/step - accuracy: 0.4941 - loss: 29.4257
Epoch 10/28
27/27 - 0s - 10ms/step - accuracy: 0.4459 - loss: 20.1128
Epoch 11/28
27/27 - 0s - 10ms/step - accuracy: 0.4532 - loss: 39.3366
Epoch 12/28
27/27 - 0s - 9ms/step - accuracy: 0.4503 - loss: 36.5297
Epoch 13/28
27/27 - 0s - 9ms/step - accuracy: 0.4500 - loss: 33.1434
Epoch 14/28
27/27 - 0s - 9ms/step - accuracy: 0.4728 - loss: 34.7000
Epoch 15/28
27/27 - 0s - 9ms/step - accu

In [85]:
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([stations[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([stations[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])


In [86]:
# Evaluate
# Reshape X_test to match the model's expected input shape
X_test_reshaped = X_test.values.reshape(-1, 15, 9)

# Convert y_test to one-hot for the confusion matrix
y_test_one_hot = to_categorical(y_test, num_classes=15)

print(confusion_matrix(y_test_one_hot, model.predict(X_test_reshaped)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 975us/step
Pred        BELGRADE
True                
BASEL           3682
BELGRADE        1092
BUDAPEST         214
DEBILT            82
DUSSELDORF        29
HEATHROW          82
KASSEL            11
LJUBLJANA         61
MAASTRICHT         9
MADRID           458
MUNCHENB           8
OSLO               5
STOCKHOLM          4
VALENTIA           1
